# Visier Python SDK - Interactive Walkthrough

This notebook provides a step-by-step guide to using the Visier Python SDK to query employee data.

## Learning Objectives

By the end of this walkthrough, you will understand how to:
- Configure the Visier Python SDK
- Authenticate with Visier Platform
- Build and execute data queries
- Process and analyze query results

## Prerequisites

- Python 3.9 or higher
- Visier Platform account with API access
- API credentials configured in `.env` file

## Step 1: Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [2]:
import sys
import warnings
import json
from datetime import datetime
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', message='.*urllib3.*NotOpenSSLWarning.*')
warnings.filterwarnings('ignore', message='.*urllib3 v2 only supports OpenSSL.*')

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    print('✅ Loaded environment variables from .env file')
except ImportError:
    print('⚠️ python-dotenv not installed. Using system environment variables.')

# Import Visier SDK components
from visier_platform_sdk import ApiClient, Configuration, DataQueryApi
from visier_platform_sdk.models import ListQueryExecutionDTO
from visier_platform_sdk.exceptions import ServiceException, ApiException, BadRequestException, UnauthorizedException, ApiValueError

import pandas as pd

print('✅ All imports successful!')

✅ Loaded environment variables from .env file
✅ All imports successful!


## Step 2: Verify Configuration

Let's check that all required environment variables are set.

In [3]:
# Check required environment variables
required_vars = [
    'VISIER_HOST',
    'VISIER_APIKEY',
    'VISIER_VANITY',
    'VISIER_USERNAME',
    'VISIER_PASSWORD'
]

missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print('❌ Missing required environment variables:')
    for var in missing_vars:
        print(f'   - {var}')
    print('\nPlease set these in your .env file or environment.')
    print('See visier.env.example for a template.')
else:
    print('✅ All required environment variables are set!')
    print(f"   Host: {os.getenv('VISIER_HOST')}")
    print(f"   Vanity: {os.getenv('VISIER_VANITY')}")
    print(f"   Username: {os.getenv('VISIER_USERNAME')}")

✅ All required environment variables are set!
   Host: https://css9xazus3demo7.api.visier.io
   Vanity: css9xazus3demo7
   Username: laktest


## Step 3: Create Configuration

The `Configuration` object holds all your Visier API credentials. The SDK can automatically read these from environment variables using `Configuration.from_env()`.

**What this does:**
- Reads VISIER_HOST, VISIER_APIKEY, VISIER_VANITY from environment
- Reads VISIER_USERNAME and VISIER_PASSWORD for basic authentication
- Creates a configuration object ready to use

In [4]:
# Create configuration from environment variables
config = Configuration.from_env()

# Set as default configuration (optional, but recommended)
Configuration.set_default(config)

print('✅ Configuration created successfully!')
print(f'   Host: {config.host}')
print(f'   Vanity: {config.vanity}')
print(f'   API Key: {"*" * 20} (hidden)')
print(f'   Username: {config.username}')

✅ Configuration created successfully!
   Host: https://css9xazus3demo7.api.visier.io
   Vanity: css9xazus3demo7
   API Key: ******************** (hidden)
   Username: laktest


## Step 4: Create API Client

The `ApiClient` is your connection to the Visier API. It handles:
- Automatic authentication (happens on first API call)
- Request formatting
- Response parsing
- Token management

**Key Point:** Authentication happens automatically when you make your first API call. You don't need to manually authenticate!

In [5]:
# Create the API client
api_client = ApiClient(config)

print('✅ API client created!')
print('   (Authentication will happen automatically on first API call)')

✅ API client created!
   (Authentication will happen automatically on first API call)


## Step 5: Create Data Query API

The `DataQueryApi` provides methods to query data from Visier. The main methods are:
- `list()`: Execute a list query (returns detailed records)
- `aggregate()`: Execute an aggregation query (returns summaries)
- `snapshot()`: Execute a snapshot query (time series data)

We'll use `list()` to get detailed employee records.

In [6]:
# Create DataQueryApi instance
data_query_api = DataQueryApi(api_client)

print('✅ DataQueryApi ready!')

✅ DataQueryApi ready!


## Step 6: Build the Query

A query defines what data you want to retrieve. It has several components:

1. **Source**: Which analytic object to query (e.g., 'Employee')
2. **Columns**: Which properties to retrieve
3. **Time Interval**: What time period to query
4. **Options**: Pagination, limits, etc.

Let's build a query for employee data with three properties:
- EmployeeID
- Time_in_Role
- Span_Of_Control

In [7]:
# Define the query as a dictionary
# This is easier to read and modify than building DTOs directly
query_dict = {
    'source': {'analyticObject': 'Employee'},
    'columns': [
        {
            'columnName': 'Employee ID',
            'columnDefinition': {
                'property': {
                    'name': 'Employee.EmployeeID',
                    'qualifyingPath': 'Employee'
                }
            }
        },
        {
            'columnName': 'Time in Role',
            'columnDefinition': {
                'property': {
                    'name': 'Employee.Time_in_Role',
                    'qualifyingPath': 'Employee'
                }
            }
        },
        {
            'columnName': 'Span of Control',
            'columnDefinition': {
                'property': {
                    'name': 'Employee.Span_Of_Control',
                    'qualifyingPath': 'Employee'
                }
            }
        }
    ],
    'timeInterval': {'fromInstant': '1735689600000'},  # December 1, 2024
    'options': {'limit': 10000, 'page': 0}
}

# Convert dictionary to JSON string, then create DTO
json_string = json.dumps(query_dict)
query_dto = ListQueryExecutionDTO.from_json(json_string)

print('✅ Query built successfully!')
print(f"   Analytic Object: {query_dict['source']['analyticObject']}")
print(f"   Properties: {len(query_dict['columns'])} columns")
print(f"   Time Period: From December 1, 2024")
print(f"   Limit: {query_dict['options']['limit']} records")

✅ Query built successfully!
   Analytic Object: Employee
   Properties: 3 columns
   Time Period: From December 1, 2024
   Limit: 10000 records


## Step 7: Execute the Query

Now we'll send the query to Visier and retrieve the data. The SDK handles all the HTTP communication, authentication, and response parsing for us.

**What happens:**
1. SDK sends the query to Visier API
2. Authentication happens automatically (if not already authenticated)
3. Visier processes the query
4. Response is returned as a `ListResponse` object

In [8]:
# Execute the query
print('🔧 Executing query...')
print('   (This may take a few seconds...)')

response = data_query_api.list(query_dto)

print('✅ Query executed successfully!')
print(f'   Response type: {type(response).__name__}')

if hasattr(response, 'rows') and response.rows:
    print(f'   Records returned: {len(response.rows)}')
else:
    print('   No rows in response')

🔧 Executing query...
   (This may take a few seconds...)
✅ Query executed successfully!
   Response type: ListResponse
   Records returned: 4349


## Step 8: Process the Response

The response comes back in a structured format with numeric keys. We need to:
1. Extract the header (column names)
2. Extract the rows (data)
3. Map numeric keys to column names
4. Convert to a pandas DataFrame for easier analysis

**Response Structure:**
```python
{
    'header': {'0': 'Employee ID', '1': 'Time in Role', '2': 'Span of Control'},
    'rows': [
        {'0': 'Employee-123', '1': 22, '2': 0},
        ...
    ]
}
```

In [9]:
# Convert response to dictionary if needed
if hasattr(response, 'to_dict'):
    data = response.to_dict()
elif hasattr(response, '__dict__'):
    data = {}
    if hasattr(response, 'header'):
        data['header'] = response.header
    if hasattr(response, 'rows'):
        data['rows'] = response.rows
else:
    data = response

# Extract header and rows
header = data.get('header', {})
rows = data.get('rows', [])

print(f'✅ Extracted {len(rows)} rows with {len(header)} columns')
print(f"   Column names: {list(header.values())[:3]}...")

✅ Extracted 4349 rows with 3 columns
   Column names: ['Employee ID', 'Time in Role', 'Span of Control']...


/Users/lakshman.rajanathan/Desktop/Cursor/PythonSDK/venv/lib/python3.8/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `Any` but got `str` with value `'Employee ID'` - serialized value may not be as expected
  Expected `Any` but got `str` with value `'Time in Role'` - serialized value may not be as expected
  Expected `Any` but got `str` with value `'Span of Control'` - serialized value may not be as expected
  Expected `Any` but got `str` with value `'Employee-6034'` - serialized value may not be as expected
  Expected `Any` but got `int` with value `22` - serialized value may not be as expected
  Expected `Any` but got `int` with value `0` - serialized value may not be as expected
  Expected `Any` but got `str` with value `'Employee-6566'` - serialized value may not be as expected
  Expected `Any` but got `int` with value `13` - serialized value may not be as expected
  Expected `Any` but got `int` with value `0` - serialized value may not be

In [10]:
# Create mapping from numeric keys to column names
column_mapping = {
    str(k): header[str(k)] 
    for k in sorted([int(k) for k in header.keys()])
}

# Convert rows to list of dictionaries with proper column names
df_rows = []
for row in rows:
    df_row = {
        column_mapping.get(str(k), k): row.get(str(k))
        for k in sorted([int(k) for k in row.keys()])
    }
    df_rows.append(df_row)

# Create DataFrame
df = pd.DataFrame(df_rows)

# Reorder columns to match original header order
column_order = [
    column_mapping[str(k)]
    for k in sorted([int(k) for k in header.keys()])
]
df = df[column_order]

print('✅ Data converted to DataFrame!')
print(f'   Shape: {df.shape[0]} rows, {df.shape[1]} columns')
print(f'   Columns: {list(df.columns)}')

✅ Data converted to DataFrame!
   Shape: 4349 rows, 3 columns
   Columns: ['Employee ID', 'Time in Role', 'Span of Control']


## Step 9: Display Results

Now let's view the data we retrieved!

In [11]:
# Display first 20 rows
print('=' * 70)
print('Query Results')
print('=' * 70)
print(f'\nFound {len(df)} employee record(s):\n')

display(df.head(20))

Query Results

Found 4349 employee record(s):



,Employee ID,Time in Role,Span of Control
0,Employee-6034,22,0
1,Employee-6566,13,0
2,Employee-7350,1,0
3,Employee-1211,37,0
4,Employee-92,59,10
5,Employee-1743,59,0
6,Employee-5383,26,0
7,Employee-7217,4,0
8,Employee-3563,59,0
9,Employee-2562,22,2


## Step 10: Basic Analysis

Now that we have the data, let's do some basic analysis!

In [12]:
# Basic statistics
print('📊 Basic Statistics:')
print('=' * 70)

if 'Time in Role' in df.columns:
    print(f'\nTime in Role:')
    print(f'   Mean: {df["Time in Role"].mean():.1f} months')
    print(f'   Median: {df["Time in Role"].median():.1f} months')
    print(f'   Min: {df["Time in Role"].min()} months')
    print(f'   Max: {df["Time in Role"].max()} months')

if 'Span of Control' in df.columns:
    print(f'\nSpan of Control:')
    print(f'   Mean: {df["Span of Control"].mean():.1f} direct reports')
    print(f'   Employees with direct reports: {(df["Span of Control"] > 0).sum()}')
    print(f'   Employees with 10+ direct reports: {(df["Span of Control"] >= 10).sum()}')

📊 Basic Statistics:

Time in Role:
   Mean: 27.2 months
   Median: 23.0 months
   Min: 0 months
   Max: 59 months

Span of Control:
   Mean: 1.0 direct reports
   Employees with direct reports: 1379
   Employees with 10+ direct reports: 85


## Summary and Next Steps

### What We Learned

1. **Configuration**: Use `Configuration.from_env()` to load credentials from environment variables
2. **API Client**: The `ApiClient` handles authentication automatically
3. **Query Structure**: Queries define source, columns, time period, and options
4. **Execution**: Use `DataQueryApi.list()` to execute list queries
5. **Processing**: Convert responses to DataFrames for easier analysis

### Next Steps

- Try modifying the query to get different properties
- Experiment with different time periods
- Explore other API methods (`aggregate()`, `snapshot()`)
- Add filters to the query
- Export data to CSV or Excel

### Resources

- [Visier Python SDK Documentation](https://github.com/visier/python-sdk)
- [Visier API Reference](https://documentation.visier.com/)
- [Visier API Samples](https://github.com/visier/api-samples)

In [ ]:
# Optional: Save to CSV
# df.to_csv('employee_data.csv', index=False)
# print('✅ Data saved to employee_data.csv')